In [3]:
!git clone -b prune-experiment https://github.com/markusloecher/TreeSandBox.git

Cloning into 'TreeSandbox'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 36 (delta 9), reused 36 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 67.83 KiB | 1.11 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [4]:
%cd /content/TreeSandbox

/content/TreeSandbox


In [5]:
!pip install -e . --verbose

Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Obtaining file:///content/TreeSandbox
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-u0vlrprs/AugmentedHierarchicalShrinkage.egg-info
  writing /tmp/pip-pip-egg-info-u0vlrprs/AugmentedHierarchicalShrinkage.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-u0vlrprs/AugmentedHierarchicalShrinkage.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-u0vlrprs/AugmentedHierarchicalShrinkage.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-u0vlrprs/AugmentedHierarchicalShrinkage.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-u0vlrprs/AugmentedHierarchicalShrinkage.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-pip-egg-info-u0vlrprs/AugmentedHierarchicalShrinkage.egg-info/SOURCES.txt'
  writing manifest file '/tmp/pip-pip-egg-info-u0vlrprs/AugmentedHierarchicalShrinka

In [14]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
#from sklearn.metrics import accuracy_score
from TreeModelsFromScratch.RandomForest import RandomForest
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

In [15]:
def print_node_info(node):
    print(f"Node ID: {node.id}")
    print(f"Samples: {node.samples}")
    print(f"Sample Indices: {node.sample_indices}")
    print(f"Class Value Distribution: {node.clf_value_dis}")
    print(f"Class Probability Distribution: {node.clf_prob_dis}")
    print(f"Gini Impurity: {node.gini}\n")

    if not node.is_leaf_node():
        print("\nLeft Child Node Info:")
        print_node_info(node.left)

        print("\nRight Child Node Info:")
        print_node_info(node.right)


# Small dataset
X_train = np.array([[2.5, 3.0],
                    [1.5, 2.5],
                    [3.5, 3.5],
                    [3.0, 2.0],
                    [2.0, 1.5]])
y_train = np.array([0, 1, 0, 1, 0])

# Create and fit a RandomForest
rf = RandomForest(n_trees=3, max_depth=2, random_state=42)
rf.fit(X_train, y_train)

# Choose a sample to add to the tree
new_sample_index = 0
new_sample_x = X_train[new_sample_index]
new_sample_y = y_train[new_sample_index]

# Traverse and add the new sample to each tree
for tree in rf.trees:
    print(f"\nBefore adding new sample to tree with root node {tree.root.id}:\n")
    print_node_info(tree.root)
    tree.traverse_add_path(new_sample_x, new_sample_index, new_sample_y)
    print(f"\nAfter adding new sample to tree with root node {tree.root.id}:\n")
    print_node_info(tree.root)


Before adding new sample to tree with root node 0:

Node ID: 0
Samples: 5
Sample Indices: [0 1 2 3 4]
Class Value Distribution: [4, 1]
Class Probability Distribution: [0.8 0.2]
Gini Impurity: 0.31999999999999984


Left Child Node Info:
Node ID: 1
Samples: 3
Sample Indices: [1 2 3]
Class Value Distribution: [3, 0]
Class Probability Distribution: [1. 0.]
Gini Impurity: 0.0


Right Child Node Info:
Node ID: 2
Samples: 2
Sample Indices: [0 4]
Class Value Distribution: [1, 1]
Class Probability Distribution: [0.5 0.5]
Gini Impurity: 0.5


Left Child Node Info:
Node ID: 3
Samples: 1
Sample Indices: [0]
Class Value Distribution: [0, 1]
Class Probability Distribution: [0. 1.]
Gini Impurity: 0.0


Right Child Node Info:
Node ID: 4
Samples: 1
Sample Indices: [4]
Class Value Distribution: [1, 0]
Class Probability Distribution: [1. 0.]
Gini Impurity: 0.0


After adding new sample to tree with root node 0:

Node ID: 0
Samples: 6
Sample Indices: [0 1 2 3 4]
Class Value Distribution: [1, 0]
Class Pro

In [16]:
import unittest
import numpy as np
from TreeModelsFromScratch.DecisionTree import DecisionTree
from TreeModelsFromScratch.RandomForest import RandomForest

class TestRandomForest(unittest.TestCase):

    def setUp(self):
        # Small dataset
        self.X_train = np.array([[2.5, 3.0],
                                 [1.5, 2.5],
                                 [3.5, 3.5],
                                 [3.0, 2.0],
                                 [2.0, 1.5]])
        self.y_train = np.array([0, 1, 0, 1, 0])
        self.rf = RandomForest(n_trees=3, max_depth=2, random_state=42)

    def test_fit(self):
        self.rf.fit(self.X_train, self.y_train)
        self.assertEqual(len(self.rf.trees), 3)
        self.assertEqual(self.rf.trees[0].root.samples, len(self.X_train))

    def test_bootstrap_sampling(self):
        X_inbag, y_inbag, idxs_inbag = self.rf._bootstrap_samples(self.X_train, self.y_train, bootstrap=True, random_state=self.rf.random_state_)
        self.assertEqual(len(idxs_inbag), len(self.X_train))
        self.assertTrue(np.all(np.isin(idxs_inbag, np.arange(len(self.X_train)))))

    def test_oob_samples(self):
        X_inbag, y_inbag, idxs_inbag = self.rf._bootstrap_samples(self.X_train, self.y_train, bootstrap=True, random_state=self.rf.random_state_)
        X_oob, y_oob, idxs_oob = self.rf._oob_samples(self.X_train, self.y_train, idxs_inbag)
        self.assertEqual(len(np.setdiff1d(np.arange(len(self.X_train)), idxs_inbag)), len(idxs_oob))

    def test_tree_growth(self):
        self.rf.fit(self.X_train, self.y_train)
        for tree in self.rf.trees:
            self.assertTrue(tree.max_depth is not None)
            self.assertTrue(tree.min_samples_split <= len(self.X_train))

    def test_shap_values(self):
        self.rf.fit(self.X_train, self.y_train)
        shap_values = self.rf.trees[0].predict_proba(self.X_train)
        self.assertEqual(shap_values.shape[0], len(self.X_train))

    def test_random_state_consistency(self):
        rf1 = RandomForest(n_trees=3, max_depth=2, random_state=42)
        rf2 = RandomForest(n_trees=3, max_depth=2, random_state=42)
        rf1.fit(self.X_train, self.y_train)
        rf2.fit(self.X_train, self.y_train)
        self.assertEqual(rf1.trees[0].root.samples, rf2.trees[0].root.samples)

# Run the tests
unittest.main(argv=[''], verbosity=2, exit=False)

import pytest
pytest.main(args=['-q', '--tb=short'])

test_bootstrap_sampling (__main__.TestRandomForest) ... ok
test_fit (__main__.TestRandomForest) ... ok
test_oob_samples (__main__.TestRandomForest) ... ok
test_random_state_consistency (__main__.TestRandomForest) ... ok
test_shap_values (__main__.TestRandomForest) ... ok
test_tree_growth (__main__.TestRandomForest) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.074s

OK



no tests ran in 0.01s


<ExitCode.NO_TESTS_COLLECTED: 5>